In [1]:
import logging,sys
log = logging.getLogger()
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler(sys.stdout))

In [2]:
from mnist import *
from convvisual import *
from layers import DilationSeparate2DLayer,DilationMerge2DLayer
import lasagne
import matplotlib.pyplot as plt
%matplotlib inline
from numpy.random import RandomState
import test_dilation

%load_ext autoreload
%autoreload 2

Using gpu device 0: GeForce GTX 780 (CNMeM is disabled, cuDNN 5005)


In [3]:
init_theano()

In [4]:
rng = RandomState(5)

n_layers = rng.randint(8)
shape_input = rng.randint(100,500,size=(2,1))
filters = rng.randint(1,4,size=(n_layers,1))
filter_size = rng.randint(1,5,size=(n_layers,2))
pools = rng.randint(1,4,size=(n_layers,2))
strides = rng.randint(1,3,size=(n_layers,2))

l_in = lasagne.layers.InputLayer((None,1,shape_input[0,0],shape_input[1,0]))
l_last = l_in
for i in np.arange(n_layers):
    l_next = lasagne.layers.Conv2DLayer(l_last,
                            num_filters=filters[i,0],
                            filter_size=list(filter_size[i,:]),
                            nonlinearity=lasagne.nonlinearities.elu)
    l_last = lasagne.layers.Pool2DLayer(l_next,
                                    pool_size=list(pools[i,:]),
                                    stride=list(strides[i,:]))
    
l_out = lasagne.layers.Conv2DLayer(l_last,
                                   num_filters=4,
                                   filter_size=(2,4))


In [5]:
from braindecode.veganlasagne.layer_util import print_layers
layers = lasagne.layers.get_all_layers(l_out)
print_layers(layers)

 0-InputLayer                                                         (None, 1, 306, 289)
 1-Conv2DLayer              2x1                       elu             (None, 3, 305, 289)
 2-Pool2DLayer              1x3 ::2 ::1               max             (None, 3, 153, 287)
 3-Conv2DLayer              1x4                       elu             (None, 3, 153, 284)
 4-Pool2DLayer              1x1 ::2 ::1               max             (None, 3, 77, 284)
 5-Conv2DLayer              3x4                       elu             (None, 1, 75, 281)
 6-Pool2DLayer              2x2 ::2 ::2               max             (None, 1, 37, 140)
 7-Conv2DLayer              2x4                       rectify         (None, 4, 36, 137)



In [6]:
X = rng.randint(100,500,size=(1,1,shape_input[0,0],shape_input[1,0]))
X = np.asarray(X,dtype=np.float32)
output = lasagne.layers.get_output(l_out,X)

/home/hartmank/braindecode/vienv/local/lib/python2.7/site-packages/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode=self.mode,
/home/hartmank/braindecode/vienv/local/lib/python2.7/site-packages/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode=self.mode,
/home/hartmank/braindecode/vienv/local/lib/python2.7/site-packages/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode=self.mode,


In [7]:
l_deconv = receptive_field_build_deconv_layers(l_out,layers[1])

(None, 1, 37, 140)
(37, 140)
(None, 1, 75, 281)
(75, 281)
o (None, 1, 37, 140)
o (None, 1, 37, 140)
(None, 3, 77, 284)
(77, 284)
o (None, 1, 75, 281)
o (None, 1, 75, 281)
(None, 3, 153, 284)
(153, 284)
o (None, 3, 77, 284)
o (None, 3, 77, 284)
(None, 3, 153, 287)
(153, 287)
o (None, 3, 153, 284)
o (None, 3, 153, 284)
(None, 3, 305, 289)
(305, 289)
o (None, 3, 153, 287)
o (None, 3, 153, 287)
(None, 1, 306, 289)
(306, 289)
o (None, 3, 305, 289)
o (None, 3, 305, 289)


In [8]:
deconv_layers = lasagne.layers.get_all_layers(l_deconv)
print_layers(deconv_layers)
assert(np.array_equal(deconv_layers[-1].output_shape,layers[0].shape))

o (None, 1, 37, 140)
o (None, 1, 37, 140)
o (None, 1, 37, 140)
o (None, 1, 75, 281)
o (None, 1, 75, 281)
o (None, 1, 75, 281)
o (None, 3, 77, 284)
o (None, 3, 77, 284)
o (None, 3, 77, 284)
o (None, 3, 153, 284)
o (None, 3, 153, 284)
o (None, 3, 153, 284)
o (None, 3, 153, 287)
o (None, 3, 153, 287)
o (None, 3, 153, 287)
o (None, 3, 305, 289)
o (None, 3, 305, 289)
o (None, 3, 305, 289)
o (None, 1, 306, 289)
o (None, 1, 306, 289)
o (None, 1, 306, 289)
 0-InputLayer                                                         (None, 4, 36, 137)
 1-TransposedConv2DLayer    2x4                       rectify         (None, 1, 37, 140)
 2-TransposedPool2DLayer    2x2 ::2 ::2               rectify         (None, 1, 75, 281)
 3-TransposedConv2DLayer    3x4                       rectify         (None, 3, 77, 284)
 4-TransposedPool2DLayer    1x1 ::2 ::1               rectify         (None, 3, 153, 284)
 5-TransposedConv2DLayer    1x4                       rectify         (None, 3, 153, 287)
 6-Transpos

In [39]:
l_in = lasagne.layers.InputLayer((None,3,8,8))
l_c1 = lasagne.layers.Conv2DLayer(l_in,
                            num_filters=3,
                            filter_size=(3,3),
                            stride=(2,2),
                            pad='full',
                            W=lasagne.init.Constant(1),
                            nonlinearity=lasagne.nonlinearities.elu)
l_p1 = lasagne.layers.Pool2DLayer(l_c1,
                                pool_size=(3,3),
                                stride=(2,2),
                                ignore_border=False)
l_shuffle = lasagne.layers.DimshuffleLayer(l_p1,(0,1,3,2))
l_r1 = lasagne.layers.ReshapeLayer(l_shuffle,([0],1,1,-1))
#l_b1 = lasagne.layers.BatchNormLayer(l_r1)
#l_d1 = lasagne.layers.DropoutLayer(l_b1)
#l_n1 = lasagne.layers.NonlinearityLayer(l_b1)
l_out = l_r1


In [40]:
layers = lasagne.layers.get_all_layers(l_out)
print_layers(layers)

 0-InputLayer                                                         (None, 3, 8, 8)
 1-Conv2DLayer              3x3 ::2 ::2               elu             (None, 3, 5, 5)
 2-Pool2DLayer              3x3 ::2 ::2               max             (None, 3, 2, 2)
 3-DimshuffleLayer                                                   
 4-ReshapeLayer                                                       (None, 1, 1, 12)



In [41]:
X = np.zeros((1,3,7,7),dtype=np.float32)
X[0,0,0,0] = 1
#X[0,0,2,3] = 1
X[0,0,5,6] = 1
print X

Y = lasagne.layers.get_output(layers,X)

[[[[ 1.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  1.]
   [ 0.  0.  0.  0.  0.  0.  0.]]

  [[ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]]

  [[ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]]]]


In [42]:
#print(Y.shape)
print Y
for out in Y:
    print out.eval()[0]
res = Y[4].eval()

[TensorConstant{[[[[ 1.  0..0.  0.]]]]}, Elemwise{switch,no_inplace}.0, Pool{ignore_border=False, mode='max', ndim=2}.0, InplaceDimShuffle{0,1,3,2}.0, Reshape{4}.0]
[[[ 1.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  1.]
  [ 0.  0.  0.  0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]]]
[[[ 1.  1.  0.  0.  0.]
  [ 1.  1.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  1.  1.]
  [ 0.  0.  0.  0.  0.]]

 [[ 1.  1.  0.  0.  0.]
  [ 1.  1. 

In [43]:
l_deconv = receptive_field_build_deconv_layers(l_out,layers[1])
layers_deconv = lasagne.layers.get_all_layers(l_deconv)
print_layers(layers_deconv)

[[0], 3, 2, 2]
(None, 3, 5, 5)
(5, 5)
(None, 3, 8, 8)
(8, 8)
o (None, 3, 5, 5)
o (None, 3, 5, 5)
o (None, 3, 5, 5)
o (None, 3, 5, 5)
o (None, 3, 5, 5)
o (None, 3, 8, 8)
o (None, 3, 8, 8)
o (None, 3, 8, 8)
 0-InputLayer                                                         (None, 1, 1, 12)
 1-ReshapeLayer                                                       (None, 3, 2, 2)
 2-DimshuffleLayer                                                   
 3-TransposedPool2DLayer    3x3 ::2 ::2               rectify         (None, 3, 5, 5)
 4-TransposedConv2DLayer    3x3 ::2 ::2               rectify         (None, 3, 8, 8)



In [44]:
Units_on = res>0
Units_on = Units_on.astype(np.float32)
Units_off = res<=0
Units_off = Units_off.astype(np.float32)
print Units_on

[[[[ 1.  0.  0.  1.  1.  0.  0.  1.  1.  0.  0.  1.]]]]


In [45]:
Y_on = lasagne.layers.get_output(layers_deconv,Units_on)
Y_off = lasagne.layers.get_output(layers_deconv,Units_off)

None <type 'NoneType'> False
a [5 5] (5, 5)
test2
(8, 8) <type 'tuple'> False
a [8 8] (8, 8)
None <type 'NoneType'> False
a [5 5] (5, 5)
test2
(8, 8) <type 'tuple'> False
a [8 8] (8, 8)


In [38]:
for out_on,out_off in zip(Y_on,Y_off):
    res_on = out_on.eval()[0]
    res_off = out_off.eval()[0]
    res2 = (res_on>0) & (res_off<=0)
    res2 = res2.astype(np.float32)
    print(res2)
    
print(res_on)

[[[ 1.  0.  0.  1.  1.  0.  0.  1.  1.  0.  0.  1.]]]
[[[ 1.  0.]
  [ 0.  1.]]

 [[ 1.  0.]
  [ 0.  1.]]

 [[ 1.  0.]
  [ 0.  1.]]]
[[[ 1.  0.]
  [ 0.  1.]]

 [[ 1.  0.]
  [ 0.  1.]]

 [[ 1.  0.]
  [ 0.  1.]]]
[[[ 1.  1.  0.  0.  0.]
  [ 1.  1.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  1.  1.]
  [ 0.  0.  0.  1.  1.]]

 [[ 1.  1.  0.  0.  0.]
  [ 1.  1.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  1.  1.]
  [ 0.  0.  0.  1.  1.]]

 [[ 1.  1.  0.  0.  0.]
  [ 1.  1.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  1.  1.]
  [ 0.  0.  0.  1.  1.]]]
[[[ 1.  1.  0.  0.  0.  0.  0.  0.]
  [ 1.  1.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  1.  1.  1.]
  [ 0.  0.  0.  0.  0.  1.  1.  1.]
  [ 0.  0.  0.  0.  0.  1.  1.  1.]]

 [[ 1.  1.  0.  0.  0.  0.  0.  0.]
  [ 1.  1.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.

In [199]:
l_in = lasagne.layers.InputLayer((None,1,2,10))
l_dil = DilationSeparate2DLayer(l_in,(1,3),((0,0),(0,2)))
l_mrg = DilationMerge2DLayer(l_dil,(1,3),((0,0),(0,2)))

layers_mrg = lasagne.layers.get_all_layers(l_mrg)
print_layers(layers_mrg)

 0-InputLayer                                                         (None, 1, 2, 10)
 1-DilationSeparate2DLayer                                            (None, 1, 2, 4)
 2-DilationMerge2DLayer                                               (None, 1, 2, 10)



In [200]:
X = np.zeros((2,1,2,10),dtype=np.float32)
X[0,0,0,:] = np.arange(1,11)
X[0,0,1,:] = np.arange(11,21)
X[1,0,0,:] = np.arange(21,31)
X[1,0,1,:] = np.arange(31,41)
print X

Y = lasagne.layers.get_output(l_mrg,X)

[[[[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.]
   [ 11.  12.  13.  14.  15.  16.  17.  18.  19.  20.]]]


 [[[ 21.  22.  23.  24.  25.  26.  27.  28.  29.  30.]
   [ 31.  32.  33.  34.  35.  36.  37.  38.  39.  40.]]]]
((0, 0), (0, 2))
Shape.0
[2 2 2 1] [2 2 2 1]
((0, 0), (0, 2))
((0, 0), (0, 0), (0, 0), (0, 2))


In [201]:
print(Y.eval())

[[[[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.]
   [ 11.  12.  13.  14.  15.  16.  17.  18.  19.  20.]]]


 [[[ 21.  22.  23.  24.  25.  26.  27.  28.  29.  30.]
   [ 31.  32.  33.  34.  35.  36.  37.  38.  39.  40.]]]]


In [4]:
#test_dilation.test_stride_reshape_layer()
test_dilation.test_inverse_dilation_layer()

 0-InputLayer                                                         [None, 1, 15, 1]
 1-StrideReshapeLayer           ::2 ::1                               (None, 1, 8, 1)
 2-StrideReshapeLayer           ::2 ::1                               (None, 1, 4, 1)
 3-FinalReshapeLayer                                                  (None, 1, 15, 1)

(None, 1, 4, 1) (None, 1, 4, 1)
stride 2 8.0 8 0.0
(None, 1, 8, 1) (None, 1, 8, 1)
stride 2 16.0 15 1.0
(None, 1, 15, 1) [None, 1, 15, 1]
 0-InputLayer                                                         (None, 1, 15, 1)
 1-InverseFinalReshapeLayer                                           (None, 1, 4, 1)
 2-DilationMerge2DLayer                                               (None, 1, 8, 1)
 3-DilationMerge2DLayer                                               (None, 1, 15, 1)
 4-FinalReshapeLayer                                                 

[array([[[[  1.],
         [  2.],
         [  3.],
         [  4.],
         [  5.],
         [  

In [7]:
test_dilation.test_dilation_layer()

 0-InputLayer                                                         [None, 1, 15, 1]
 1-StrideReshapeLayer           ::2 ::1                               (None, 1, 8, 1)
 2-StrideReshapeLayer           ::2 ::1                               (None, 1, 4, 1)
 3-FinalReshapeLayer                                                  (None, 1, 15, 1)

 0-InputLayer                                                         [None, 1, 15, 1]
 1-StrideReshapeLayer           ::2 ::1                               (None, 1, 8, 1)
 2-StrideReshapeLayer           ::2 ::1                               (None, 1, 4, 1)
 3-FinalReshapeLayer                                                  (None, 1, 15, 1)
 4-DilationSeparate2DLayer                                            (None, 1, 8, 1)
 5-DilationSeparate2DLayer                                            (None, 1, 4, 1)
 6-DilationMerge2DLayer                                               (None, 1, 15, 1)

[array([[[[  1.],
         [  2.],
         [  